In [7]:
import torch # a Tensor library like NumPy, with strong GPU support
import torch.optim as optim # functions related to optimization algorithms

import torch.nn as nn # a neural networks library deeply integrated with autograd designed for maximum flexibility
import torch.nn.functional as F
from torch import no_grad

# nn and nn.functional works kinda similarly. If you are interested in their subtile difference, please check out this discussion https://discuss.pytorch.org/t/what-is-the-difference-between-torch-nn-and-torch-nn-functional/33597

from torch.utils.data import DataLoader # utility functions such as DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import cv2

import pandas as pd
import numpy as np
import os
import pickle
%matplotlib inline

In [8]:
torch.cuda.is_available()

True

In [9]:
np.set_printoptions(suppress=True)

trainingdata = torchvision.datasets.FashionMNIST('./FashionMNIST/',train=True,download=True,transform=torchvision.transforms.ToTensor())
testdata = torchvision.datasets.FashionMNIST('./FashionMNIST/',train=False,download=True,transform=torchvision.transforms.ToTensor())

train_dataLoader= DataLoader(trainingdata,batch_size=64,shuffle=True)
val_dataloader= DataLoader(testdata,batch_size=5000)
images,labels=next(iter(train_dataLoader))

In [10]:
val_test_data=list(iter(val_dataloader))
val_images,val_labels=val_test_data[0]

test_images,test_labels=val_test_data[1]
test_images=test_images.reshape(-1,784)

val_images=val_images.reshape(-1,784)

model=nn.Sequential(
    nn.Linear(784,128),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(64,10),
    nn.LogSoftmax(dim=1)
)

In [11]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)
#load data and model to gpu

model.to('cuda')
val_images,val_labels=val_images.cuda(),val_labels.cuda()
EPOCHS = 25

train_loss_history= []
val_loss_history= []

train_acc_history= []
val_acc_history= []

for epoch in range(EPOCHS):
    start_time = time.time() # record start time
    running_loss = 0
    running_acc = 0
    
    for images,labels in train_dataLoader:
        images,labels=images.cuda(),labels.cuda()   
        
        X=images.view(-1,784)
        
        
        torch.save(model, './model_'+str(epoch)+'.pt')

        optimizer.zero_grad()
        output=model.forward(X)
        loss=criterion(output,labels)
        
        result=torch.argmax(output,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))
        
        loss.backward()
        optimizer.step()

    else:
        train_loss=running_loss/len(train_dataLoader)
        train_acc=running_acc/len(train_dataLoader)
        
        with no_grad():
            model.eval()
            output=model.forward(val_images)
            val_loss=criterion(output,val_labels)
            model.train()
        
        result_labels=torch.argmax(output,dim=1)
        accuracy=torch.mean((result_labels==val_labels).type(torch.float))
        
        train_loss_history.append(train_loss)
        val_loss_history.append(val_loss.item())
        
        train_acc_history.append(train_acc.item())
        val_acc_history.append(accuracy.item())
        
        end_time = time.time()
        minute, second = epoch_time(start_time, end_time)
    
        print(f'Epoch: {epoch+1}/{EPOCHS}') 
        print(f'Training Loss: {train_loss}.. Validation Loss: {val_loss}')
        print(f'Training Accuracy: {train_acc}.. Validtion Accuracy: {accuracy}')
        print(f'Time Elapsed: {minute} in minute.. {second} in second')
        print('')
        

Epoch: 1/25
Training Loss: 0.6404233677650312.. Validation Loss: 0.5186417102813721
Training Accuracy: 0.7671575546264648.. Validtion Accuracy: 0.817799985408783
Time Elapsed: 0 in minute.. 16 in second

Epoch: 2/25
Training Loss: 0.5610688782926562.. Validation Loss: 0.5176253914833069
Training Accuracy: 0.8015725016593933.. Validtion Accuracy: 0.7989999651908875
Time Elapsed: 0 in minute.. 12 in second

Epoch: 3/25
Training Loss: 0.5339445419657205.. Validation Loss: 0.458041250705719
Training Accuracy: 0.8143823742866516.. Validtion Accuracy: 0.8405999541282654
Time Elapsed: 0 in minute.. 12 in second

Epoch: 4/25
Training Loss: 0.5199949301954018.. Validation Loss: 0.4639351963996887
Training Accuracy: 0.8175473213195801.. Validtion Accuracy: 0.8389999866485596
Time Elapsed: 0 in minute.. 14 in second

Epoch: 5/25
Training Loss: 0.5171977080611277.. Validation Loss: 0.46013906598091125
Training Accuracy: 0.8227112293243408.. Validtion Accuracy: 0.8509999513626099
Time Elapsed: 0 in